# Creating a State: Introduction to QuantumState, state representations, and associated visualization

## Overview [Public API]


Our quantum states are represented by objects of a `QuantumState` class. This is a wrapper class which contains one or more circuit representations; in the future, this class is also expected to mediate conversions between different state representations.

Our quantum states have 3 currently planned representations:
1. **Density matrix:** this is the most powerful representation, and can represent mixed quantum states and the most types of noise. However, the size of the matrix scales exponentially with the number of qubits.
2. **Stabilizer:** less powerful than the density matrix (cannot represent mixed states), but simulation of a circuit with stabilizer formalism has a polynomial runtime, and thus it can access much larger states
3. **Graph:** this is a graph-based representation of quantum states (it can only represent a class of graphs called "graph states"). While more restrictive than the stabilizer representation, it may allow us to use graph-theoretic tools (e.g. graph edit distance) in our solvers.

We will consider each state representation first.

## Density Matrix State Representation

**Relevant python files:** graphiq/backends/density_matrix/state.py

This is not an overview of density matrix formalism in quantum mechanics, but rather of our density matrix class.

Internally, the density matrix is represented by a numpy array. Additionally, the class contains helper functions which allows us to apply quantum operations (e.g. unitary gates, measurements) or noise to our system without manually modifying the numpy array.

### Initializing `DensityMatrix` state representation [Public API]

Most often, the only direct state manipulation a user will have to perform is the initialization of their ideal state (the remaining state manipulation occurs during the compilation/simulation of the circuit, which is done automatically).

In [3]:
""" Initializing a density matrix of size n, with every qubit in state |0> """

from graphiq.backends.density_matrix.state import DensityMatrix

n = 3
dm_state = DensityMatrix(n)

# We can print the density matrix
print(dm_state.data)

# Or we can view it via matplotlib... Though there seems to be an unfortunate non-visible square that happens to be the one we want
dm_state.draw()

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/_libs/interval.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/_libs/interval.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/_libs/interval.cpython-310-darwin.so' (no such file), '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/_libs/interval.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [ ]:
""" Initializing a density matrix from a numpy array """

# We can initialize it from any defined numpy array of adequate dimensions
import numpy as np

n = 4
data = np.eye(2**n)  # this generates an identity matrix for an n-qubit state
dm_state = DensityMatrix(data)

print(dm_state.data)

dm_state.draw()

In [ ]:
""" Initializing a state with density matrix helper functions """

# Some states can be more difficult to build from scratch.
# Our code base provides helper functions for density matrix numpy manipulation, which can be used as needed
import graphiq.backends.density_matrix.functions as dmf

# 4-qubit GHZ state: |phi> = 1/sqrt(2) * (|0000> + |1111>)
ghz4_dm = dmf.ket2dm(
    (dmf.tensor(4 * [dmf.state_ketz0()]) + dmf.tensor(4 * [dmf.state_ketz1()]))
    / np.sqrt(2)
)

ghz4_state = DensityMatrix(ghz4_dm)
ghz4_state.draw()

In [ ]:
""" 
Defining a target state from a graph by converting the target graph to a density matrix

When the target state is a graph state, it tends to be easiest to express them as graphs. Those can then be converted
a density matrix.
"""

import networkx as nx  # this is a graph-manipulation python library. We use it for our DAG circuits as well

linear_cluster_3_graph = nx.Graph([(1, 2), (2, 3)])  # Edge between node 1-2, node 2-3
nx.draw(linear_cluster_3_graph)

linear_cluster_3_state = DensityMatrix.from_graph(linear_cluster_3_graph)
linear_cluster_3_state.draw()

### Further transformations on the `DensityMatrix` state representation [General API]

When working on the compiler (or possibly when initializing a more complicated target state, the additional functionalities of the `DensityMatrix` class may become useful.

In [ ]:
""" Apply a unitary transformation """

# Suppose that we want to simulate the following circuit: a single qubit, a single gate
from graphiq.circuit.circuit_dag import CircuitDAG
from graphiq.circuit.ops import Hadamard

c = CircuitDAG()
c.add(Hadamard(register=0, reg_type="e"))
c.draw_circuit()

In [ ]:
# Before any gates are applied, the density matrix data are the default values when the DensityMatrix is initialized by an integer n
n = 1

dm_state = DensityMatrix(n)
dm_state.draw()

In [ ]:
# We can get gates from the density matrix helper functions
#
h_gate = dmf.get_one_qubit_gate(
    n,  # number of total qubits in the state
    0,  # index of the density matrix which represents the target qubit
    dmf.hadamard(),  # the single-qubit state version of the gate we want to apply
)
dm_state.apply_unitary(h_gate)
dm_state.draw()

In [ ]:
""" Measure a qubit (projects the qubit onto a given basis--we will focus on the computational basis Z)"""

# We measure qubit to convert the (inaccessible to us) quantum information to classical information
projectors = dmf.projectors_zbasis(n, 0)  # this is the basis on which we project
outcome = dm_state.apply_measurement(
    projectors,  # the projector
    measurement_determinism="probabilistic"  # most quantum measurements are inherently probabilistic,
    # but we allow deterministic measurements for algorithmic reasons
)
print(f"This time, the outcome was: {outcome}")
dm_state.draw()

**NOTE:** We can also apply a quantum channel to the state, but this is mostly used for noise in our application so we will not discuss it here

## Stabilizer State Preresentation

**Relevant python files:** graphiq/backends/stabilizer/state.py, graphiq/backends/stabilizer/function*

The Stabilizer represent n-qubit with 4 vectors:
- Destabilizer: n vector
- Stabilizer: n vector
- phase: 2n vector
- i phase: 2n vector

### Initializing `Stabilizer` state representation [Public API]

The Stabilizer class has the `__str__` function that helps view the data of the class through the `print()` function. 

In [ ]:
""" Initializing a stabilizer of size n, with every qubit in state |0> """

from graphiq.backends.stabilizer.state import Stabilizer

n = 3
stabilizer_state = Stabilizer(n)
print(stabilizer_state)

In [ ]:
""" Initializing a stabilizer from a Clifford Tableau """

from graphiq.backends.stabilizer.clifford_tableau import CliffordTableau

n = 3
tableau = CliffordTableau(n)

stabilizer_state = Stabilizer(tableau)
print(stabilizer_state)

In [ ]:
""" 
Defining a target state from a graph by converting the target graph to a stabilizer
"""

import networkx as nx  
from graphiq.backends.stabilizer.functions.rep_conversion import get_clifford_tableau_from_graph

linear_cluster_3_graph = nx.Graph([(1, 2), (2, 3)])  # Edge between node 1-2, node 2-3
nx.draw(linear_cluster_3_graph)

linear_cluster_3_stabilizer_state = get_clifford_tableau_from_graph(linear_cluster_3_graph)
print(linear_cluster_3_stabilizer_state)

### Further transformations on the `Stabilizer` state representation [General API]

The transformation functionality for `Stabilizer` class is very useful. Allows users to apply operations to the qubits and transform the stabilizer to the correct state

In [ ]:
from graphiq.circuit.circuit_dag import CircuitDAG
from graphiq.circuit.ops import Hadamard

c = CircuitDAG()
c.add(Hadamard(register=0, reg_type="e"))
c.draw_circuit()

In [ ]:
# Before any gates are applied 
n = 1

stabilizer_state = Stabilizer(n)
print(stabilizer_state)

In [ ]:
"""Apply a unitary transformation and measurement"""

# Apply Hadamard gate on the first qubit
stabilizer_state = Stabilizer(1)
stabilizer_state.apply_hadamard(0)

print(stabilizer_state)

In [ ]:
from graphiq.backends.stabilizer.functions.clifford import measure_z

# Apply measurement on z basis
# By default the measurement determinism is probabilistic
outcome = measure_z(stabilizer_state.tableau, 0, measurement_determinism="probabilistic")
print(f"Outcome of the measurement: {outcome}")

## Graph State Representation (In development)

**Relevant python files:** graphiq/backends/graph/state.py, graphiq/backends/graph/function.py

The graph state does not have a fully implemented compiler yet, there is no tutorial for graph state at this time.

## QuantumState representation [General API]

**Relevant python files:** graphiq/state.py

At this time, the QuantumState representation is quite bare-bones. However, a lot of planned functionality will be added to it moving forward.

I would suggest looking at the code and `TODO` statements. For now, we will only go over initialization.

**NOTE:** All data representations which are valid initializers for `DensityMatrix`, `Stabilizer` or `Graph` are valid initialization data for `QuantumState`

In [ ]:
""" Initializing a single representation """
from graphiq.state import QuantumState

n = 2
data = np.eye(2**n)

state = QuantumState(
    data,  # initialization data
    rep_type="dm",  # can also be 'g' for graph or 's' for stabilizer
)

print(state.rep_data.data)

## Conversion between state representations

Not all representations can be converted to one another without loss of information.
* The `Graph` representation can represent only a class of quantum states called graph states, and is the least general. 
* The `Stabilizer` representation is more general, and can represent any "pure states" (there are no unknowns associated with classical probabilities).
* The `DensityMatrix` representation is most general of all, and can represent both pure states and mixed states.

However, we still have functions to convert between all these states (albeit imperfectly in some cases--and the limitations are important to understand).

Note that these functions act on the underlying representations of different states (i.e. numpy array, networkx Graph), not on the state objects themselves.

In [ ]:
""" Graph to density matrix"""
import graphiq.backends.state_rep_conversion as rc
import src.visualizers.density_matrix as vdm

linear3_g = nx.Graph([(1, 2), (2, 3)])  # Edge between node 1-2, node 2-3
linear3_dm = rc.graph_to_density(linear3_g)

nx.draw(linear3_g)
vdm.density_matrix_bars(linear3_dm)

In [ ]:
""" Density matrix to graph: exact """

# we know that linear3_dm is a graph state--we can convert it back to a graph
graph_list = rc.density_to_graph(linear3_dm, threshold=0.4999)
# as linear3_dm is a pure state, the graph_list should contain only one element
linear3_graph_from_dm = nx.from_numpy_matrix(graph_list[0][1])
nx.draw(linear3_graph_from_dm)

In [ ]:
# CAREFUL. density_to_graph is not an exact conversion for reasons explained above
# If we change the threshold (even by a very small amount), the resulting graph may change
graph_list = rc.density_to_graph(linear3_dm, threshold=0.5)
linear3_graph_from_dm2 = nx.from_numpy_matrix(
    graph_list[0][1]
)
nx.draw(linear3_graph_from_dm2)

In [ ]:
""" Density matrix to graph, inexact """

# Let's consider something that is NOT a graph state now (e.g. the GHZ4 state)
ghz4_dm = dmf.ket2dm(
    (dmf.tensor(4 * [dmf.state_ketz0()]) + dmf.tensor(4 * [dmf.state_ketz1()]))
    / np.sqrt(2)
)

# No edge seems to be added here, no matter how low we drive the threshold
ghz4_g_from_dm1 = nx.from_numpy_matrix(rc.density_to_graph(ghz4_dm, threshold=0.00001))
nx.draw(ghz4_g_from_dm1)

In [ ]:
""" Graph to stabilizer and back """
linear3_stabilizer = rc.graph_to_stabilizer(linear3_g)
print(linear3_stabilizer)
nx.draw(rc.stabilizer_to_graph(linear3_stabilizer))

In [ ]:
""" Stabilizer to density and back """
linear3_dm_from_stabilizer = rc.stabilizer_to_density(linear3_stabilizer)
vdm.density_matrix_bars(linear3_dm_from_stabilizer)
linear3_stabilizer_from_dm = rc.density_to_stabilizer(linear3_dm_from_stabilizer)
print(linear3_stabilizer_from_dm)